ArcPro
1. Create an ND dataset
	1. Edit properties to fit
2. Add points layer for drops and start
	1. Probably have to geocode addresses
3. Create a VRP layer (it will reference ND in contents)
	1. Add locations via import orders w/ times and map, max violation time
	2. Add start/stop via import depots w/
	3. Add routes (make a lot of assumptions/interpretations)
4. Set properties for VRP layer
5. Directions in directions pane


In [9]:
#import layers to GDB
import arcpy

#create a target dataset in the GDB
arcpy.CreateFeatureDataset_management("Lab3.gdb", "Streets", 4326)
arcpy.conversion.FeatureClassToGeodatabase("Data\shp_trans_road_centerlines_gac\RoadCenterline.shp",
                                           "Lab3.gdb\Streets")

#make the

<Result 'Lab3.gdb\\Streets'>

Build Network Dataset

In [54]:
arcpy.na.CreateNetworkDataset(r"Lab3.gdb\Streets",
                              "Streets_ND","RoadCenterline",
                               "ELEVATION_FIELDS")
#can further custom configure in ND properties pages in ArcPro, but not here
#create a new f
#arcpy.na.BuildNetwork("Lab3.gdb\Streets\Streets_ND")
#SPEED_IMP/ROUTESPEED = speedlimit

<Result 'Lab3.gdb\\Streets\\Streets_ND'>

Convert addresses to points layers

In [48]:
# Model Environment settings
scratchWorkspace=r"C:\Users\Cole\Documents\GitHub\GIS5572\Lab3\Lab3.gdb",
workspace=r"C:\Users\Cole\Documents\GitHub\GIS5572\Lab3\Lab3.gdb"

Address = "C:\\Users\\Cole\\Documents\\GitHub\\GIS5572\\Lab3\\Data\\AddressSpreadsheet.csv"
Depot = "C:\\Users\\Cole\\Documents\\GitHub\\GIS5572\\Lab3\\Data\\DepotSpreadsheet.csv"

ArcGIS_World_Geocoding_Service = "https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/ArcGIS World Geocoding Service"

# Process: Geocode Addresses (Geocode Addresses) (geocoding)
AddressGeocoderRetry = "C:\\Users\\Cole\\Documents\\GitHub\\GIS5572\\Lab3\\Lab3.gdb\\AddressGeocoderRetry"
arcpy.geocoding.GeocodeAddresses(in_table=Address, address_locator=ArcGIS_World_Geocoding_Service, in_address_fields="'Address or Place' 'Street Address' VISIBLE NONE;Address2 <None> VISIBLE NONE;Address3 <None> VISIBLE NONE;Neighborhood <None> VISIBLE NONE;City City VISIBLE NONE;County <None> VISIBLE NONE;State State VISIBLE NONE;ZIP ZIP VISIBLE NONE;ZIP4 <None> VISIBLE NONE;Country <None> VISIBLE NONE", out_feature_class=AddressGeocoderRetry, out_relationship_type="STATIC", country=["US"], location_type="ROUTING_LOCATION", category=["Subaddress", "Point Address", "Street Address", "Distance Marker", "Intersection", "Street Name"])

# Model Environment settings

# Process: Geocode Addresses (Geocode Addresses) (geocoding)
DepotGeocoderRetry = "C:\\Users\\Cole\\Documents\\GitHub\\GIS5572\\Lab3\\Lab3.gdb\\DepotGeocoderRetry"
arcpy.geocoding.GeocodeAddresses(in_table=Depot, address_locator=ArcGIS_World_Geocoding_Service, in_address_fields="'Address or Place' 'Street Address' VISIBLE NONE;Address2 <None> VISIBLE NONE;Address3 <None> VISIBLE NONE;Neighborhood <None> VISIBLE NONE;City City VISIBLE NONE;County <None> VISIBLE NONE;State State VISIBLE NONE;ZIP ZIP VISIBLE NONE;ZIP4 <None> VISIBLE NONE;Country <None> VISIBLE NONE", out_feature_class=DepotGeocoderRetry, out_relationship_type="STATIC", country=["US"], location_type="ROUTING_LOCATION", category=["Subaddress", "Point Address", "Street Address", "Distance Marker", "Intersection", "Street Name"])

<Result 'C:\\Users\\Cole\\Documents\\GitHub\\GIS5572\\Lab3\\Lab3.gdb\\DepotGeocoderRetry'>

Make VRP Layer and Solve

In [53]:
# Import system modules
import arcpy
import os

output_dir = r"C:\Users\Cole\Documents\GitHub\GIS5572\Lab3"
arcpy.env.workspace = os.path.join(output_dir, "Lab3.gdb")
arcpy.env.overwriteOutput = True

# Set local variables, ND
input_gdb = r"C:\Users\Cole\Documents\GitHub\GIS5572\Lab3\Lab3.gdb"
network = os.path.join(input_gdb, "Streets\Streets_ND")
layer_name = "StoreDeliveryRoute"
output_layer_file = os.path.join(output_dir, layer_name + ".lyrx")

#VPR layer basic specs
travel_mode = "Driving Time"
time_units = "Minutes"
distance_units = "Miles"

#create the VRP layer output
result_object = arcpy.na.MakeVehicleRoutingProblemAnalysisLayer(
network, layer_name, travel_mode, time_units, distance_units,
line_shape="STRAIGHT_LINES")

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 030253: Network has no travel modes.
Failed to execute (MakeVehicleRoutingProblemAnalysisLayer).


In [ ]:
# Get the layer object form the result object. The route layer can now be referenced using the layer object.
VRP_object = result_object.getOutput(0)

# Get the names of all the sublayers within the VRP layer.
sub_layer_names = arcpy.na.GetNAClassNames(VRP_object)

In [ ]:
############################################################
in_orders = os.path.join(input_gdb,"Orders")

orders_layer_name = sub_layer_names["Orders"]

#map the fields from orders to correct properties/values
candidate_fields = arcpy.ListFields(in_orders)
order_field_mappings = arcpy.na.NAClassFieldMappings(layer_object, orders_layer_name, False, candidate_fields)
order_field_mappings["TimeWindowStart"].mappedFieldName = "TimeStart1"
order_field_mappings["TimeWindowEnd"].mappedFieldName = "TimeEnd1"
order_field_mappings["DeliveryQuantity_1"].mappedFieldName = "Demand"
order_field_mappings["MaxViolationTime"].defaultValue = 0

#add the locations to the VRP
arcpy.na.AddLocations(layer_object, orders_layer_name, in_orders, order_field_mappings, "")

In [ ]:
##########################################################    
in_depots = os.path.join(input_gdb, "DistributionCenter")

depots_layer_name = sub_layer_names["Depots"]
    
#map the fields from depots to correct properties/values
depot_field_mappings = arcpy.na.NAClassFieldMappings(layer_object, depots_layer_name)
depot_field_mappings["Name"].mappedFieldName = "Name"
depot_field_mappings["TimeWindowStart"].defaultValue = "8 AM"
depot_field_mappings["TimeWindowEnd"].defaultValue = "6 PM"

#add the depots to the VRP
arcpy.na.AddLocations(layer_object, depots_layer_name, in_depots, depot_field_mappings, "")

In [ ]:
#######################################################################
# Load the routes from a table containing information about routes. In this
# case, since the fields on the routes table and property names for Routes
# are the same, we will just use the default field mappings

#routes_layer_name = sub_layer_names["Routes"]

#map the fields from routes to correct properties/values (most are defaults)
#arcpy.na.AddVehicleRoutingProblemRoutes(layer_object, 2,
 #                                       "UPSTruck", {start_depot_name},
  #                                      {end_depot_name}, "8AM","8AM",
   #                                     10,{capacities}, {route_constraints},
    #                                    {costs},{additional_route_time},
     #                                   "CLEAR")


#routes_field_mappings = arcpy.na.NAClassFieldMappings(layer_object, routes_layer_name)
#routes_field_mappings["Name"].mappedFieldName = "Name"
#routes_field_mappings["StartDepotName"].mappedFieldName = "StartDepotName"
#routes_field_mappings["EndDepotName"].mappedFieldName = "EndDepotName"
#routes_field_mappings["StartDepotServiceTime"].mappedFieldName = "StartDepotServiceTime"
#routes_field_mappings["Capacity_1"].mappedFieldName = "Capacities"
#routes_field_mappings["CostPerUnitTime"].mappedFieldName = "CostPerUnitTime"
#routes_field_mappings["CostPerUnitDistance"].mappedFieldName = "CostPerUnitDistance"
#routes_field_mappings["MaxOrderCount"].mappedFieldName = "MaxOrderCount"
#routes_field_mappings["MaxTotalTime"].mappedFieldName = "MaxTotalTime"
#routes_field_mappings["MaxTotalTravelTime"].mappedFieldName = "MaxTotalTravelTime"
#routes_field_mappings["MaxTotalDistance"].mappedFieldName = "MaxTotalDistance"
#arcpy.na.AddLocations(layer_object, routes_layer_name, in_routes, routes_field_mappings, "")

# Solve the VRP layer
#arcpy.na.Solve(layer_object)

# Save the solved VRP layer as a layer file on disk with relative paths
#arcpy.management.SaveToLayerFile(layer_object, output_layer_file, "RELATIVE")